In [26]:
# Purchase Pattern Analytics - Market Basket Analysis (Apriori)

In [28]:
import os
os.listdir()


['.ipynb_checkpoints', 'clientproject1.csv', 'Purchase_Pattern_Apriori.ipynb']

In [8]:
df = pd.read_csv("clientproject1.csv", encoding="latin1")
df.head()

C:\Users\priya\AppData\Local\Temp\ipykernel_33308\2040283746.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("clientproject1.csv", encoding="latin1")


,BillNo,Itemname,Quantity,Present_Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8.0,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [7]:
df.columns

Index(['BillNo', 'Itemname', 'Quantity', 'Present_Date', 'Price', 'CustomerID',
       'Country'],
      dtype='object')

In [37]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   BillNo        522064 non-null  object 
 1   Itemname      520609 non-null  object 
 2   Quantity      522064 non-null  float64
 3   Present_Date  522063 non-null  object 
 4   Price         522064 non-null  object 
 5   CustomerID    388023 non-null  float64
 6   Country       522063 non-null  object 
dtypes: float64(2), object(5)
memory usage: 27.9+ MB


In [39]:
df = df.rename(columns={
    'BillNo': 'InvoiceNo',
    'Itemname': 'Description',
    'Quantity': 'Quantity'
})

df.head()


,InvoiceNo,Description,Quantity,Present_Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8.0,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [40]:
# Remove missing Invoice or Product names
df = df.dropna(subset=['InvoiceNo', 'Description'])

# Convert InvoiceNo to string
df['InvoiceNo'] = df['InvoiceNo'].astype(str)

# Remove cancelled invoices (if any)
df = df[~df['InvoiceNo'].str.startswith('C')]

# Keep only positive quantity
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df = df[df['Quantity'] > 0]

df.head()


,InvoiceNo,Description,Quantity,Present_Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8.0,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6.0,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [44]:
transactions = (
    df.groupby('InvoiceNo')['Description']
      .apply(list)
      .tolist()
)

transactions[:5]


[['WHITE HANGING HEART T-LIGHT HOLDER',
  'WHITE METAL LANTERN',
  'CREAM CUPID HEARTS COAT HANGER',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  'RED WOOLLY HOTTIE WHITE HEART.',
  'SET 7 BABUSHKA NESTING BOXES',
  'GLASS STAR FROSTED T-LIGHT HOLDER'],
 ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT'],
 ['ASSORTED COLOUR BIRD ORNAMENT',
  "POPPY'S PLAYHOUSE BEDROOM",
  "POPPY'S PLAYHOUSE KITCHEN",
  'FELTCRAFT PRINCESS CHARLOTTE DOLL',
  'IVORY KNITTED MUG COSY',
  'BOX OF 6 ASSORTED COLOUR TEASPOONS',
  'BOX OF VINTAGE JIGSAW BLOCKS',
  'BOX OF VINTAGE ALPHABET BLOCKS',
  'HOME BUILDING BLOCK WORD',
  'LOVE BUILDING BLOCK WORD',
  'RECIPE BOX WITH METAL HEART',
  'DOORMAT NEW ENGLAND'],
 ['JAM MAKING SET WITH JARS',
  'RED COAT RACK PARIS FASHION',
  'YELLOW COAT RACK PARIS FASHION',
  'BLUE COAT RACK PARIS FASHION'],
 ['BATH BUILDING BLOCK WORD']]

In [15]:
# one-hot encoding
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(te_array, columns=te.columns_)

basket.head()


,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
# Count how many times each product appears
item_freq = basket.sum()

# Keep only products bought at least 100 times
basket_reduced = basket[item_freq[item_freq >= 100].index]

basket_reduced.shape


(19735, 1531)

In [21]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(
    basket_reduced,
    min_support=0.02,   # increase support
    use_colnames=True
)

frequent_itemsets.head()


,support,itemsets
0,0.023157,(3 STRIPEY MICE FELTCRAFT)
1,0.023512,(4 TRADITIONAL SPINNING TOPS)
2,0.047732,(6 RIBBONS RUSTIC CHARM)
3,0.021130,(60 CAKE CASES DOLLY GIRL DESIGN)
4,0.030403,(60 CAKE CASES VINTAGE CHRISTMAS)


In [22]:
frequent_itemsets.sort_values(by='support', ascending=False).head(10)


,support,itemsets
276,0.111578,(WHITE HANGING HEART T-LIGHT HOLDER)
104,0.104586,(JUMBO BAG RED RETROSPOT)
199,0.096478,(REGENCY CAKESTAND 3 TIER)
159,0.083912,(PARTY BUNTING)
129,0.078085,(LUNCH BAG RED RETROSPOT)
15,0.072511,(ASSORTED COLOUR BIRD ORNAMENT)
220,0.068204,(SET OF 3 CAKE TINS PANTRY DESIGN)
151,0.064809,(PACK OF 72 RETROSPOT CAKE CASES)
122,0.063846,(LUNCH BAG BLACK SKULL.)
141,0.062427,(NATURAL SLATE HEART CHALKBOARD)


In [23]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(
    frequent_itemsets,
    metric='lift',
    min_threshold=1
)

rules[['antecedents','consequents','support','confidence','lift']].head()


,antecedents,consequents,support,confidence,lift
0,(PACK OF 72 RETROSPOT CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.022042,0.340109,8.411103
1,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETROSPOT CAKE CASES),0.022042,0.545113,8.411103
2,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.020826,0.541502,11.051232
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.020826,0.425026,11.051232
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.032075,0.616358,12.578936


In [24]:
strong_rules = rules[
    (rules['confidence'] >= 0.6) &
    (rules['lift'] > 1)
]

strong_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.052040,0.048999,0.032075,0.616358,12.578936,1.0,0.029525,2.478878,0.971034,0.465099,0.596592,0.635480
5,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.048999,0.052040,0.032075,0.654602,12.578936,1.0,0.029525,2.744544,0.967930,0.465099,0.635641,0.635480
10,(CHARLOTTE BAG PINK POLKADOT),(RED RETROSPOT CHARLOTTE BAG),0.037395,0.052090,0.026349,0.704607,13.526673,1.0,0.024401,3.208979,0.962048,0.417335,0.688374,0.605222
18,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.042057,0.044794,0.026602,0.632530,14.121020,1.0,0.024719,2.599414,0.969978,0.441548,0.615298,0.613211
20,(DOTCOM POSTAGE),(JUMBO BAG RED RETROSPOT),0.035875,0.104586,0.024576,0.685028,6.549919,1.0,0.020824,2.842840,0.878856,0.212068,0.648239,0.460004
24,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.038206,0.045858,0.027565,0.721485,15.733165,1.0,0.025813,3.425826,0.973639,0.487892,0.708100,0.661295
25,(GARDENERS KNEELING PAD KEEP CALM),(GARDENERS KNEELING PAD CUP OF TEA),0.045858,0.038206,0.027565,0.601105,15.733165,1.0,0.025813,2.411145,0.981447,0.487892,0.585259,0.661295
26,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037294,0.049405,0.030656,0.822011,16.638343,1.0,0.028814,5.340750,0.976308,0.547016,0.812760,0.721262
27,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.049405,0.037294,0.030656,0.620513,16.638343,1.0,0.028814,2.536860,0.988746,0.547016,0.605812,0.721262
30,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.051330,0.049405,0.037041,0.721619,14.606308,1.0,0.034505,3.414727,0.981940,0.581543,0.707151,0.735681


In [27]:
strong_rules.to_csv("apriori_results.csv", index=False)
